In [1]:
import lsst.alert.packet as packet
import os
import numpy as np
import pandas as pd
import urllib.request
import scipy as sp
import sys
# sys.path.append('/home/s/seanmacb/LSST-Camera/Extras and Utilities/pkgs')
# sys.path.append('/home/s/seanmacb/LSST-Camera/Extras and Utilities/pkgs/eo_pipe/python/lsst/eo/pipe')
# import eo_pipe.python.lsst.eo.pipe as eo_pipe
# from eo_pipe.python.lsst.eo.pipe import (readNoiseTask, darkCurrentTask, defectsTask, eperTask, divisaderoTearingTask, ptcPlotsTask,linearityPlotsTask, bfAnalysisTask)
import copy
import lsst.daf.butler as daf_butler
import lsst.afw.math as afw_math
from matplotlib import pyplot as plt
from lsst.ip.isr import IsrTask, IsrTaskConfig
plt.style.use("seaborn-v0_8-talk")
from matplotlib import lines
from mpl_toolkits import axes_grid1
from astropy.stats import sigma_clip
from scipy.stats import skew
import astropy
from tabulate import tabulate
from scipy.stats import norm


from lsst.obs.lsst import LsstCam, LsstTS8
from IPython.display import display_html, HTML
# import myutils_ar.myutils as myu

from astropy.time import Time
from collections import defaultdict
from datetime import datetime, timedelta
from scipy.stats import norm

# LSST Specific Libraries
from lsst_efd_client import EfdClient
from lsst.summit.utils.plotting import plot
from lsst.summit.utils.blockUtils import BlockParser
from lsst.summit.utils.efdUtils import makeEfdClient, getEfdData
from lsst.summit.utils.tmaUtils import (
    getCommandsDuringEvent,
    TMAEvent,
    TMAEventMaker,
    TMAState,
)
import matplotlib.dates as mdates

async def query_bump_logs_in_chunks(
    start_date,
    end_date,
    client_name="",
    chunk_size_days=3,
    topic_name="lsst.sal.MTM1M3.logevent_logMessage",
    fields=["message"],
):
    """
    Queries the log messages related to bump tests from the EFD in chunks.

    Args:
        start_date (str): Start date of the query in ISO format (YYYY-MM-DD).

        end_date (str): End date of the query in ISO format (YYYY-MM-DD).

        client_name (str, optional): Name of the EFD client. Defaults to "".

        chunk_size_days (int, optional): Number of days per chunk. Defaults to 3.

        topic_name (str, optional): SAL topic name to be queried by the client. Defaults to lsst.sal.MTM1M3.logevent_logMessage.

        fields (list[str], optional): Fields to be queried by the client. Defaults to ["message"].

    Returns:
        pandas.DataFrame: Concatenated DataFrame containing the queried log messages.
    """

    client = makeClient(client_name)

    # Convert start and end dates to datetime objects
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)

    # Initialize an empty DataFrame to store concatenated results
    all_data = pd.DataFrame()

    current_start = start
    while current_start < end:
        current_end = min(current_start + timedelta(days=chunk_size_days), end)
        try:
            # Query the data for the current chunk
            chunk_data = await client.select_time_series(
                topic_name=topic_name,
                fields=fields,
                start=Time(current_start.isoformat(), format="isot", scale="utc"),
                end=Time(current_end.isoformat(), format="isot", scale="utc"),
            )
            # Concatenate the chunk data to the main DataFrame
            all_data = pd.concat([all_data, chunk_data], ignore_index=False)
        except Exception as e:
            print(
                f"Error querying data from {current_start.isoformat()} to {current_end.isoformat()}: {e}"
            )
            continue  # Optionally, continue to the next chunk

        # Move to the next chunk
        current_start = current_end

    return all_data

from astropy.utils.iers import conf
conf.auto_max_age = None

def makeClient(client_name):
    # Create the client based on client_name
    if client_name == "summit_efd":
        return makeEfdClient("summit_efd")
    elif client_name == "usdf_efd":
        return makeEfdClient("usdf_efd")
    elif client_name == "idf_efd":
        return makeEfdClient("idf_efd")
    else:
        return makeEfdClient()  # Default client

import healpy as hp

# Example usage:
# begin = "2023-11-13T01:00"
# end = "2023-12-21T01:00"
# bump_logs = await query_bump_logs_in_chunks(begin, end, client_name='')

import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib') 
def showAndClear():
    plt.show()
    # Clear the current axes.
    plt.cla()
    # Clear the current figure.
    plt.clf()
    # Closes all the figure windows.
    plt.close("all")
    plt.close(fig)

    return


async def getDataFrame(client, starts, ends, topic, verbose=True, fields=None):

    all_data = pd.DataFrame()
    for start, end in zip(starts, ends):
        if verbose:
            print(
                r"Starting query for time range {} - {}".format(start, end),
                end=" . . . ",
            )
        if fields != None:
            df_bump = await client.select_time_series(
                topic, fields, Time(start), Time(end)
            )
        else:
            df_bump = await client.select_time_series(
                topic, "*", Time(start), Time(end)
            )

        all_data = pd.concat([all_data, df_bump], ignore_index=False)

        del df_bump

        if verbose:
            print("Finished")

    return all_data


def makeDateRange(startPoint, endPoint, step=np.timedelta64(1, "D")):
    starts = np.arange(startPoint, endPoint, step=step)
    ends = starts + np.timedelta64(1, "D")
    return starts, ends

def fitGaussian(data, ax):
    mu, std = norm.fit(data)

    xmin, xmax = ax.get_xlim()
    x = np.linspace(np.floor(xmin), np.ceil(xmax), int(10e4))
    p = norm.pdf(x, mu, std)

    return mu, std, p, x, xmin, xmax

def getFWHM_from_gaussian(sigma):
    return 2 * np.sqrt(np.log(2) * 2) * sigma

from astropy.time import Time
from astropy.coordinates import get_sun
import astropy.coordinates as coord
import astropy.units as u



from astropy.stats import mad_std
instrument="LSSTCam"
# embargoButler = daf_butler.Butler("/repo/embargo",collections=['LSSTCam/raw/all','LSSTCam/calib'])
# mainButler = daf_butler.Butler("/repo/main",collections=['LSSTCam/raw/all','LSSTCam/calib/unbounded'])
# flatColl = "LSSTCam/calib/DM-50448/initial-ugr-flats/flatGen-u.20250423a"
# flat_det184 = embargoButler.get("flat",collections=flatColl,detector=184,instrument=instrument,physical_filter="i_39")
isrConfig = IsrTaskConfig()
isrConfig.doLinearize=False
isrConfig.doOverscan=True
isrConfig.overscan.fitType="MEDIAN_PER_ROW"
isrConfig.overscan.doParallelOverscan=True
isrConfig.doAssembleCcd=True
isrConfig.doBias=False
isrConfig.doVariance=False
isrConfig.doCrosstalk=False
isrConfig.doBrighterFatter=False
isrConfig.doDark=False
isrConfig.doStrayLight=False
isrConfig.doFlat=False
isrConfig.doFringe=False
isrConfig.doApplyGains=True
isrConfig.usePtcGains=False
isrConfig.doDefect=False
isrConfig.doNanMasking=True
isrConfig.doInterpolate=False
isrConfig.doSaturation=False
isrConfig.doSaturationInterpolation=False
isrTask = IsrTask(config=isrConfig)

In [2]:
uri = packet.get_uri_to_latest_schema()
schem = packet.Schema.from_uri(uri)

In [3]:
schem.definition

{'type': 'record',
 'doc': 'Rubin Avro alert schema v7.4',
 'name': 'lsst.v7_4.alert',
 'fields': [{'doc': 'unique alert identifer',
   'name': 'alertId',
   'type': 'long'},
  {'name': 'diaSource',
   'type': {'type': 'record',
    'name': 'lsst.v7_4.diaSource',
    'fields': [{'doc': 'Unique identifier of this DiaSource.',
      'name': 'diaSourceId',
      'type': 'long'},
     {'doc': 'Id of the visit where this diaSource was measured.',
      'name': 'visit',
      'type': 'long'},
     {'doc': 'Id of the detector where this diaSource was measured. Datatype short instead of byte because of DB concerns about unsigned bytes.',
      'name': 'detector',
      'type': 'int'},
     {'default': None,
      'doc': 'Id of the diaObject this source was associated with, if any. If not, it is set to NULL (each diaSource will be associated with either a diaObject or ssObject).',
      'name': 'diaObjectId',
      'type': ['null', 'long']},
     {'default': None,
      'doc': 'Id of the ssObje

In [4]:
packet.simulate.simulate_alert(schem.definition)

{'alertId': 1843213837121355684,
 'diaSource': {'diaSourceId': -8778672038334143001,
  'visit': 443119330007188748,
  'detector': -2069236051,
  'diaObjectId': None,
  'ssObjectId': None,
  'parentDiaSourceId': None,
  'midpointMjdTai': 0.09204287053023008,
  'ra': 0.7537751377178844,
  'raErr': None,
  'dec': 0.7328965668208586,
  'decErr': None,
  'ra_dec_Cov': None,
  'x': 0.9964846968650818,
  'xErr': None,
  'y': 0.999973714351654,
  'yErr': None,
  'x_y_Cov': None,
  'centroid_flag': None,
  'apFlux': None,
  'apFluxErr': None,
  'apFlux_flag': None,
  'apFlux_flag_apertureTruncated': None,
  'is_negative': None,
  'snr': None,
  'psfFlux': None,
  'psfFluxErr': None,
  'psfRa': None,
  'psfRaErr': None,
  'psfDec': None,
  'psfDecErr': None,
  'psfFlux_psfRa_Cov': None,
  'psfFlux_psfDec_Cov': None,
  'psfRa_psfDec_Cov': None,
  'psfLnL': None,
  'psfChi2': None,
  'psfNdata': None,
  'psfFlux_flag': None,
  'psfFlux_flag_edge': None,
  'psfFlux_flag_noGoodPixels': None,
  'trai

In [5]:
client = makeClient("base_efd",)
start = Time("2025-08-22T20:30:16.20")
end = Time("2025-10-29T17:20")

In [6]:
# client.get_fields()
a = await client.get_topics()

In [7]:
for entry in a:
    if entry.lower().__contains__("header"):
        print(entry)

lsst.sal.ATHeaderService.ackcmd
lsst.sal.ATHeaderService.command_disable
lsst.sal.ATHeaderService.command_enable
lsst.sal.ATHeaderService.command_exitControl
lsst.sal.ATHeaderService.command_setLogLevel
lsst.sal.ATHeaderService.command_standby
lsst.sal.ATHeaderService.command_start
lsst.sal.ATHeaderService.logevent_authList
lsst.sal.ATHeaderService.logevent_errorCode
lsst.sal.ATHeaderService.logevent_heartbeat
lsst.sal.ATHeaderService.logevent_largeFileObjectAvailable
lsst.sal.ATHeaderService.logevent_logLevel
lsst.sal.ATHeaderService.logevent_logMessage
lsst.sal.ATHeaderService.logevent_simulationMode
lsst.sal.ATHeaderService.logevent_softwareVersions
lsst.sal.ATHeaderService.logevent_summaryState
lsst.sal.CCHeaderService.ackcmd
lsst.sal.CCHeaderService.command_disable
lsst.sal.CCHeaderService.command_enable
lsst.sal.CCHeaderService.command_exitControl
lsst.sal.CCHeaderService.command_standby
lsst.sal.CCHeaderService.command_start
lsst.sal.CCHeaderService.logevent_authList
lsst.sal.CC

In [8]:
def parseAlertContents(inputList,dictionary):
    """
    A function to resolve alert contents from the header file of an acquired image
    """

    dictionary["diaSource"]["ra"] = getRA(inputList)
    dictionary["diaSource"]["dec"] = getDec(inputList)
    dictionary["observation_reason"] = getObsReason(inputList)
    dictionary["target_name"] = getTargetName(inputList)
    sourceID = generateSourceID()
    dictionary["diaSourceId"] = sourceID
    dictionary["diaSource"]["diaSourceId"] = sourceID

    return dictionary

def generateSourceID():
    return np.random.randint(9223372036854775808,dtype=np.int64)

def getRA(inputList):

    raStart = float(getCoordFromKeyword("RASTART",inputList))
    raEnd = float(getCoordFromKeyword("RAEND",inputList))

    avgCenterRa = np.mean([raStart,raEnd]) # Take the average of the two for now

    if np.random.randint(0,2):
        return avgCenterRa+np.random.random()*1.75
    else:
        return avgCenterRa-np.random.random()*1.75

def getDec(inputList):

    decStart = float(getCoordFromKeyword("DECSTART",inputList))
    decEnd = float(getCoordFromKeyword("DECEND",inputList))

    avgCenterDec = np.mean([decStart,decEnd]) # Take the average of the two for now

    if np.random.randint(0,2):
        return avgCenterDec+np.random.random()*1.75
    else:
        return avgCenterDec-np.random.random()*1.75

def getObsReason(inputList):
    obsReasons = getCoordFromKeyword("REASON",inputList,returnAll=True)

    returnString = ""
    for k in obsReasons[3:]:
        returnString+=k
        if k!=obsReasons[-1]:
            returnString+=", "

    return returnString

def getTargetName(inputList):
    
    obsReasons = getCoordFromKeyword("COMMENT",inputList,returnAll=True)

    returnString = ""
    for k in obsReasons[3:]:
        returnString+=k
        if k!=obsReasons[-1]:
            returnString+=", "

    return returnString

def getCoordFromKeyword(keyword,inputList,index=-1,returnAll=False):
    msk = ["keyword: {}".format(keyword) in s for s in inputList]
    firstIndex = np.argwhere(inputList==inputList[msk][0])[0][0]
    if returnAll:
        return inputList[firstIndex+1].split(" ")
    else:
        return inputList[firstIndex+1].split(" ")[index]

In [10]:
topic = "lsst.sal.MTHeaderService.logevent_largeFileObjectAvailable"
result = (await getDataFrame(client,[np.datetime64(start.to_datetime())],[np.datetime64(end.to_datetime())],topic)).sort_index()

yml = urllib.request.urlopen(result.iloc[-1]['url']).read().decode()
fullList = np.array(yml.split("\n"))

Starting query for time range 2025-08-22T20:30:16.200000 - 2025-10-29T17:20:00.000000 . . . Finished


URLError: <urlopen error Tunnel connection failed: 503 Service Unavailable>

In [ ]:
%%time

myDictionary = {"diaSource":{}}
for k in np.arange(1000):
    resultDictionary = parseAlertContents(fullList,myDictionary)

In [ ]:
resultDictionary